In [1]:
len('GHKGSGNYRWRCKNQN')

16

In [1]:
!pip install biopython

   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.8 MB 2.4 MB/s eta 0:00:01
   ---------------------- ----------------- 1.6/2.8 MB 3.8 MB/s eta 0:00:01
   ----------------------------- ---------- 2.1/2.8 MB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 3.5 MB/s eta 0:00:00
